### Description of test task

Intorduction
This assessment is designed to help you practice your data parsing and feature engineering skills. You will
be given a dataset called data.csv that contains three columns: id, application_date, and contracts. The
contracts column is a JSON string that contains information about the contracts that a customer has
signed up for. Your task is to parse this JSON string and calculate a set of features from it.

Data

The data.csv file contains the following columns:
• id: The unique identifier for the row.
• application_date: The start date of the application.
• contracts: A JSON string that contains information about the contracts that a customer has
signed up for. You must analyze JSON data format yourself.

Deliverables

• Code/script files: Provide the script or code (e.g., Python script, Jupyter notebook) that performs
the necessary parsing and feature calculation.
• Output dataset: Share the resulting dataset in csv file that includes the calculated features as
columns. Please name it as contract_features.csv.

Additional Notes
• You can modify the assessment to calculate more features.
• You can only use python for this assessment.
• You can find the definitions of the features in the features.xlsx file.

Submission

You must share a link to a version-controlled repository (e.g., GitHub).

----

https://github.com/SaygafarovT/de_assessment

#### Importing python libraries

In [1]:
### Loading the necessary libraries for our work (some may need to be installed via pip install if they are not already available on your system)
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
from ast import literal_eval

Открываем датафрейм и проверяем данные

In [2]:
df = pd.read_csv('data.csv')
df.head()

,id,application_date,contracts
0,2925210.0,2024-02-12 19:22:46.652000+00:00,NaN
1,2925211.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
2,2925212.0,2024-02-12 19:24:41.493000+00:00,NaN
3,2925213.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
4,2925214.0,2024-02-12 19:24:56.857000+00:00,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1000 non-null   float64
 1   application_date  1000 non-null   object 
 2   contracts         495 non-null    object 
dtypes: float64(1), object(2)
memory usage: 23.6+ KB


In [4]:
# taking the data where contracts are not NaN.
df = df.dropna(subset = ['contracts']).reset_index(drop=True)
df['application_date'] = pd.to_datetime(df['application_date'], format="mixed")
df["application_date"] = df["application_date"].dt.date
app_dates = df['application_date'].to_list()
df["application_date"].value_counts()
contracts_list = df["contracts"].to_list()

In [5]:
# Checking the correctness of the claim_date field format in case of incorrect export or data entry errors.
# This is necessary to ensure the features are calculated correctly.
value_len_set = set()

for contract in contracts_list:
    contract = literal_eval(contract)
    if type(contract) == list:
        for each_c in contract:
            for k,v in each_c.items():
                if k=='claim_date':
                    value_len_set.add(len(v))

    else:
        for k,v in each_c.items():
            if k=='claim_date':
                value_len_set.add(len(v))
 
       
assert len(value_len_set) == 1, 'Уникальный формат даты будет 1 если там нет проблемных'

#### We parse the data from the "contracts" column and generate a dataframe. We also add the "application_date" field. The "application_id" is not required since it is not needed for task execution.

In [6]:
lst = []

for contracts, dates in zip(contracts_list, app_dates):
    contracts = literal_eval(contracts)
    if type(contracts) == list:
        for contract in contracts:
            contract['application_date'] = dates
            lst.append(contract)
    else:
        contract['application_date'] = dates
        lst.append(contracts)
        
df = pd.DataFrame(lst)
                

In [7]:
### cheking data
df.head()

,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date,application_date
0,522530,003,500000000,0,13.02.2020,609965,17.02.2020,2024-02-12
1,,014,,,28.08.2020,F00013731,,2024-02-12
2,,014,,,08.10.2020,F00021301,,2024-02-12
3,,014,,,25.11.2020,F00037907,,2024-02-12
4,,053,,,09.12.2020,34852,,2024-02-12


In [8]:
# Filling empty fields with NaN or None.

def from_str_to_nan(x):
    if x:
        return x
    else:
        np.nan

df = df.applymap(from_str_to_nan)

/var/folders/w6/0vybrrdx6clclv578llq4bhr0000gn/T/ipykernel_30933/185220042.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(from_str_to_nan)


In [9]:
# Checking result
df = df.drop_duplicates()
df.head()

,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date,application_date
0,522530,003,500000000.0,NaN,13.02.2020,609965,17.02.2020,2024-02-12
1,None,014,NaN,NaN,28.08.2020,F00013731,None,2024-02-12
2,None,014,NaN,NaN,08.10.2020,F00021301,None,2024-02-12
3,None,014,NaN,NaN,25.11.2020,F00037907,None,2024-02-12
4,None,053,NaN,NaN,09.12.2020,34852,None,2024-02-12


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2543 entries, 0 to 9257
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   contract_id       548 non-null    object 
 1   bank              2506 non-null   object 
 2   summa             548 non-null    float64
 3   loan_summa        166 non-null    float64
 4   claim_date        2543 non-null   object 
 5   claim_id          2543 non-null   object 
 6   contract_date     548 non-null    object 
 7   application_date  2530 non-null   object 
dtypes: float64(2), object(6)
memory usage: 178.8+ KB


In [11]:
### feature 1 - tot_claim_cnt_l180d
days_to_subtract = 180
max_date = pd.to_datetime(df["claim_date"], format="%d.%m.%Y").max()
min_date = max_date - timedelta(days=days_to_subtract)

def tot_claim_cnt_l180d(df: pd.DataFrame):
    'this function creares additional feature (tot_claim_cnt_l180d) from new formed dataframe'
    global max_date, min_date

    claim_id = df["claim_id"]
    claim_date = pd.to_datetime(df["claim_date"], format="%d.%m.%Y")
    
    if not claim_id and not claim_date:
        return "-3"
    elif not claim_date:
        return "unconsidered"

    elif claim_date >= min_date:
        return "claim"
    else:
        return "before 180"
    
    
    

df["tot_claim_cnt_l180d"] = df.apply(tot_claim_cnt_l180d, axis=1)

In [12]:
df["tot_claim_cnt_l180d"].value_counts()

tot_claim_cnt_l180d
before 180    1508
claim         1035
Name: count, dtype: int64

In [13]:
df.head()

,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date,application_date,tot_claim_cnt_l180d
0,522530,003,500000000.0,NaN,13.02.2020,609965,17.02.2020,2024-02-12,before 180
1,None,014,NaN,NaN,28.08.2020,F00013731,None,2024-02-12,before 180
2,None,014,NaN,NaN,08.10.2020,F00021301,None,2024-02-12,before 180
3,None,014,NaN,NaN,25.11.2020,F00037907,None,2024-02-12,before 180
4,None,053,NaN,NaN,09.12.2020,34852,None,2024-02-12,before 180


In [14]:
#### feature 2 - disb_bank_loan_wo_tbc

def disb_bank_loan_wo_tbc(df: pd.DataFrame):
    'this function creates new column disb_bank_loan_wo_tbc based on the given instructions'
    contract_date = df['contract_date']
    claim_id = df['claim_id']
    loan_summa = df['loan_summa']
    excluded_banks = ['LIZ', 'LOM', 'MKO', 'SUG', np.nan]
    bank = df['bank']

    if pd.isna(claim_id):
        return '-3'
    elif pd.isna(loan_summa):
        return '-1'
    elif pd.notna(contract_date) and (pd.notna(bank) and bank not in excluded_banks):
        return 'claim'

    
df['disb_bank_loan_wo_tbc'] = df.apply(disb_bank_loan_wo_tbc, axis = 1)

In [15]:
# cheking data
df['disb_bank_loan_wo_tbc'].value_counts()

disb_bank_loan_wo_tbc
-1       2377
claim     117
Name: count, dtype: int64

In [16]:
### feature 3 - day_sinlastloan

df['contract_date'] = pd.to_datetime(df['contract_date'], format="mixed")
df['application_date'] = pd.to_datetime(df['application_date'])

def day_sinlastloan(df: pd.DataFrame):
    'this function creates new column day_sinlastloan based on the given instructions'
    
    contract_date = df['contract_date']
    app_date = df['application_date']
    disb_bank_loan_wo_tbc = df['disb_bank_loan_wo_tbc']
    claim_id = df['claim_id']
    
    if pd.isna(claim_id):
        return '-3'
    
    elif disb_bank_loan_wo_tbc == 'claim':
        return app_date - contract_date    
    
    else:
        return - 1
    
    
    
    
df['day_sinlastloan'] = df.apply(day_sinlastloan, axis = 1)
df['day_sinlastloan'] = pd.to_timedelta(df['day_sinlastloan'])
df['day_sinlastloan'] = df['day_sinlastloan'].dt.days

In [17]:
### checking data
df[df['day_sinlastloan'] != -1]

,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date,application_date,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc,day_sinlastloan
440,95511,004,2.800000e+09,1.826667e+09,08.11.2022,95511,2022-08-11,2024-02-13,before 180,claim,551.0
445,414955,011,5.000000e+08,9.502356e+07,05.04.2023,2411695,2023-05-04,2024-02-13,before 180,claim,285.0
451,552855526,063,7.000000e+08,6.161615e+08,04.06.2023,552855526,2023-05-06,2024-02-13,before 180,claim,283.0
589,552933635,063,8.000000e+08,5.593000e+08,17.06.2023,552933635,2023-06-19,2024-02-13,before 180,claim,239.0
722,554074162,063,1.000000e+09,9.868236e+08,28.12.2023,554074162,2023-12-28,2024-02-13,claim,claim,47.0
...,...,...,...,...,...,...,...,...,...,...,...
9175,14605776,062,1.921000e+09,1.766564e+09,11.09.2023,14605776,2023-11-09,2024-02-13,claim,claim,96.0
9209,12144992,005,1.930000e+09,1.177375e+09,21.10.2022,3382076,2022-10-21,2024-02-13,before 180,claim,480.0
9214,553516633,063,4.600000e+08,4.227725e+08,09.10.2023,553516633,2023-09-10,2024-02-13,claim,claim,156.0
9220,79365,014,3.400000e+08,3.305556e+08,26.12.2023,79365,2023-12-26,2024-02-13,claim,claim,49.0
